In [2]:
vdw_type = "protor"

L = 5
rs = 1.4
η = 0.3665
bounds = 200.0
σ_t = 0.75
σ_r = 0.35
overlap_jump = 0.0
overlap_slope = 0.85
delaunay_eps = 10.0
comment = ""

energy_specifier = "oj_$(replace(string(overlap_jump), "." => "_"))_os_$(replace(string(overlap_slope), "." => "_"))"
simulation_time_minutes = 12.0 * 60.0

720.0

In [3]:
for file in readdir("../assets/input/jld2/2_low/")
    println(split(file, ".")[1])
end

1ei7_1
1ei7_10
1ei7_11
1ei7_12
1ei7_13
1ei7_14
1ei7_15
1ei7_16
1ei7_17
1ei7_18
1ei7_19
1ei7_2
1ei7_20
1ei7_21
1ei7_22
1ei7_23
1ei7_3
1ei7_4
1ei7_5
1ei7_6
1ei7_7
1ei7_8
1ei7_9
2tmv_1
2tmv_10
2tmv_11
2tmv_12
2tmv_13
2tmv_14
2tmv_15
2tmv_16
2tmv_17
2tmv_2
2tmv_3
2tmv_4
2tmv_5
2tmv_6
2tmv_7
2tmv_8
2tmv_9
6r7m_1
6r7m_10
6r7m_11
6r7m_12
6r7m_13
6r7m_14
6r7m_15
6r7m_16
6r7m_17
6r7m_18
6r7m_19
6r7m_2
6r7m_20
6r7m_21
6r7m_3
6r7m_4
6r7m_5
6r7m_6
6r7m_7
6r7m_8
6r7m_9
6sae_1
6sae_10
6sae_11
6sae_12
6sae_13
6sae_2
6sae_3
6sae_4
6sae_5
6sae_6
6sae_7
6sae_8
6sae_9
6sag_1
6sag_2
6sag_3
6sag_4
6sag_5
6sag_6


In [11]:
T = 2.4
ε = 0.001

input_specifier = "2_low_refinement"
template_folder = "assets/input/jld2/2_low/"

simulations_per_combination = 5


open("configs/$(input_specifier)_config.txt", "w") do io
    println(io,"ArrayTaskID input_string")
    i = 0
    for file in readdir("../$(template_folder)")
        if split(file,"_")[1] in ["6sag", "6sae"]  
            i+=1
            template_file = "$(template_folder)$(file)"
            output_directory = "assets/output/assembly/$(input_specifier)_$(energy_specifier)/"
            mol_type = split(file, ".")[1]
            for j in 1:simulations_per_combination
                name = "$(j)_$(mol_type)_$(vdw_type)_$(energy_specifier)"
                input_string = escape_string("name=\"$name\";T=$(T);rs=$rs;η=$η;L=$L;ε=$ε;σ_t=$σ_t;σ_r=$σ_r;overlap_jump=$overlap_jump;overlap_slope=$overlap_slope;simulation_time_minutes=$simulation_time_minutes;output_directory=\"$output_directory\";template_file=\"$template_file\";delaunay_eps=$delaunay_eps;comment=\"$comment\"")
                println(io, "$((i-1) * simulations_per_combination + j) $input_string")
            end
        end
    end
end


In [12]:
total_simulations = length(readlines("configs/$(input_specifier)_config.txt")) - 1

96

In [9]:
# #Ts = [2.0, 2.4, 2.8]
# #εs = [0.001, 0.003, 0.005]

# simulations_per_combination = 10
# total_simulations = length(Ts) * length(εs) * simulations_per_combination
# mol_type = "6r7m"
# input_specifier = "3_6r7m_stacked"
# open("configs/$(input_specifier)_config.txt", "w") do io
#     println(io,"ArrayTaskID input_string")
#     i = 1
#     for T in Ts
#         for ε in εs
#             i += 1
#             template_file = "assets/input/jld2/$(input_specifier).jld2"
#             output_directory = "assets/output/assembly/$(input_specifier)_$(energy_specifier)/"
#             for j in 1:simulations_per_combination
#                 name = "$((i-1)*simulations_per_combination+j)_$(mol_type)_$(vdw_type)_$(energy_specifier)"
#                 input_string = escape_string("name=\"$name\";T=$(T);rs=$rs;η=$η;L=$L;ε=$ε;σ_t=$σ_t;σ_r=$σ_r;overlap_jump=$overlap_jump;overlap_slope=$overlap_slope;simulation_time_minutes=$simulation_time_minutes;output_directory=\"$output_directory\";template_file=\"$template_file\";delaunay_eps=$delaunay_eps;comment=\"$comment\"")
#                 println(io, "$((i-1)*simulations_per_combination + j) $input_string")
#             end
#         end
#     end
# end

In [21]:
hours = Int(round(simulation_time_minutes / 60.0))
days = hours ÷ 24
remaining_hours = hours % 24
remaining_hours_string = remaining_hours < 10 ? "0$(remaining_hours)" : string(remaining_hours)
buffer_time_string = simulation_time_minutes < 5 ? "0$(Int(simulation_time_minutes)+2)" : "30"

open("$(input_specifier)_script.job", "w") do io
    println(io, "#!/bin/bash")
    println(io, "#SBATCH --job-name=SolvationSimulations")
    println(io, "#SBATCH --time=0$(days)-$(remaining_hours_string):$(buffer_time_string)")
    println(io, "#SBATCH --ntasks=1")
    println(io, "#SBATCH --cpus-per-task=1")
    println(io, "#SBATCH --array=1-$(total_simulations)")
    println(io, "#SBATCH --chdir=/work/spirandelli/MorphoMolMonteCarlo/")
    println(io, "#SBATCH -o ./hpc/out/%a.out # STDOUT")
    println(io, "")
    println(io, "export http_proxy=http://proxy2.uni-potsdam.de:3128 #Setting proxy, due to lack of Internet on compute nodes.")
    println(io, "export https_proxy=http://proxy2.uni-potsdam.de:3128")
    println(io, "")
    println(io, "module purge")
    println(io, "module load lang/Julia/1.7.3-linux-x86_64")
    println(io, "")
    println(io, "# Specify the path to the config file")
    println(io, "config=hpc/configs/$(input_specifier)_config.txt")
    println(io, "")
    println(io, "# Extract the variables from config file")
    println(io, "config_string=\$(awk -v ArrayTaskID=\$SLURM_ARRAY_TASK_ID '\$1==ArrayTaskID {print \$2}' \$config)")
    println(io, "")
    println(io, "julia -e \"$(escape_string("include(\"hpc/hmc_call.jl\"); hmc_call(\"\$config_string\")"))\"")
    println(io, "")
    println(io, "# copy back results")
    println(io, "mkdir -p ~/outputs/ && cp -r assets/output/assembly/* ~/outputs/")
end